In [1]:
# Cell 1: Imports & Environment Setup
import os
import time
import json
import boto3
from dotenv import load_dotenv

# LangChain & AI Libraries
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_aws import BedrockEmbeddings, ChatBedrock
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

# Pinecone & Evaluation
from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import BM25Encoder
# from ragas import evaluate
# from ragas.metrics import faithfulness, answer_relevancy, context_precision
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper
# from ragas.run_config import RunConfig
# from datasets import Dataset

# Load Environment Variables
load_dotenv(override=True)

print("✅ Libraries loaded. Environment verified.")

c:\Anaconda3\envs\pashdoc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries loaded. Environment verified.


In [2]:
# Cell 1.5: Initialize LangSmith Tracing
import langsmith
from langsmith import Client

# Explicitly enable tracing (this ensures it's active in notebooks)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "sbi-insurance-rag-project"

# Verify LangSmith is configured
try:
    ls_client = Client()
    print(f"✅ LangSmith Connected!")
    print(f"   Project: {os.getenv('LANGCHAIN_PROJECT')}")
    print(f"   API Key Set: {'Yes' if os.getenv('LANGCHAIN_API_KEY') else 'No'}")
except Exception as e:
    print(f"⚠️ LangSmith Connection Error: {e}")


✅ LangSmith Connected!
   Project: sbi-insurance-rag-project
   API Key Set: Yes


In [3]:
# Cell 2: Smart Initialization & Duplicate Check
# Configuration
file_path = "SBIhomeinsurance_home.pdf" # Make sure this matches your file name
index_name = "sbi-home-insurance-rag-hybrid" # Using your existing hybrid index name

# 1. Connect to Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# 2. Check if Index Exists
existing_indexes = [index.name for index in pc.list_indexes()]

if index_name not in existing_indexes:
    print(f"⚠️ Index '{index_name}' not found. Creating it...")
    pc.create_index(
        name=index_name,
        dimension=1024, # Titan v2
        metric="dotproduct", # Required for Hybrid
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    time.sleep(20) # Wait for init
    print("✅ Index created successfully.")
else:
    print(f"✅ Index '{index_name}' already exists.")

# 3. Connect to the Index
index = pc.Index(index_name)

# 4. Check if File is Already Ingested (The "Smart" Check)
# We perform a dummy query filtering by this specific source file
print(f"🔍 Checking if '{file_path}' is already in the database...")

# We use a dummy vector just to trigger the metadata filter
dummy_vector = [0.0] * 1024 
check_response = index.query(
    vector=dummy_vector,
    top_k=1,
    filter={"source": file_path},
    include_metadata=False
)

if len(check_response['matches']) > 0:
    print(f"✅ File '{file_path}' detected in Pinecone.")
    print("🚀 SKIPPING Docling & Embeddings to save cost.")
    should_ingest = False
else:
    print(f"⚠️ File '{file_path}' NOT found in Pinecone.")
    print("⚙️ Proceeding with Ingestion...")
    should_ingest = True

✅ Index 'sbi-home-insurance-rag-hybrid' already exists.
🔍 Checking if 'SBIhomeinsurance_home.pdf' is already in the database...
✅ File 'SBIhomeinsurance_home.pdf' detected in Pinecone.
🚀 SKIPPING Docling & Embeddings to save cost.


In [5]:
# Cell 3: Load & Chunk (Conditional)
final_chunks = []

if should_ingest:
    print(f"📄 Starting Docling processing for {file_path}...")
    
    # A. Load with Docling (Export to Markdown)
    loader = DoclingLoader(
        file_path=file_path,
        export_type=ExportType.MARKDOWN
    )
    docs = loader.load()
    print("✅ PDF Loaded via Docling.")

    # B. Split by Headers (Level 1)
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(docs[0].page_content)
    
    # C. Split by Size (Level 2)
    chunk_size = 1000
    chunk_overlap = 200
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    final_chunks = text_splitter.split_documents(md_header_splits)

    # D. Add Metadata Tags (Crucial for Smart Indexing)
    for chunk in final_chunks:
        chunk.metadata["source"] = file_path # Used for filtering later
        # We also keep the 'text' in metadata for Hybrid retrieval
        chunk.metadata["text"] = chunk.page_content 
    
    print(f"✅ Chunking Complete. Created {len(final_chunks)} chunks.")
    print("Sample Metadata:", final_chunks[0].metadata)

else:
    print("⏭️ Skipping Loading & Chunking (Data already exists).")

⏭️ Skipping Loading & Chunking (Data already exists).


In [6]:
# Cell 4: Hybrid Embedding & Upsert (Conditional)
import boto3
from langchain_aws import BedrockEmbeddings
from pinecone_text.sparse import BM25Encoder

# 1. Initialize AWS Bedrock Embeddings (Need this for both Ingestion AND Querying)
boto3_session = boto3.Session()
bedrock_client = boto3_session.client("bedrock-runtime", region_name="us-east-1")

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    client=bedrock_client
)

# 2. Initialize BM25 Encoder
bm25 = BM25Encoder()
bm25_filename = "bm25_values.json"

if should_ingest:
    print("⚙️ Generatings Embeddings & Upserting...")
    
    # A. Fit BM25 on the new text
    chunk_texts = [chunk.page_content for chunk in final_chunks]
    bm25.fit(chunk_texts)
    bm25.dump(bm25_filename) # Save for future use
    print("✅ BM25 Encoder fitted and saved.")
    
    # B. Generate Vectors & Upsert
    vectors_to_upsert = []
    
    print(f"Generating vectors for {len(final_chunks)} chunks...")
    for i, chunk in enumerate(final_chunks):
        # 1. Dense Vector (Titan)
        dense_vec = embeddings.embed_query(chunk.page_content)
        
        # 2. Sparse Vector (BM25)
        sparse_vec = bm25.encode_documents(chunk.page_content)
        
        # 3. Create ID (Unique based on source + index)
        # We use a simple hash or index. Here index 'i' is fine for this run.
        # Ideally, hash the text to avoid dupes, but for now:
        vector_id = f"{file_path}_{i}"
        
        vectors_to_upsert.append({
            "id": vector_id,
            "values": dense_vec,
            "sparse_values": sparse_vec,
            "metadata": chunk.metadata # Includes 'source' and 'text'
        })
        
    # C. Batch Upsert to Pinecone
    batch_size = 50
    for i in range(0, len(vectors_to_upsert), batch_size):
        batch = vectors_to_upsert[i : i + batch_size]
        index.upsert(vectors=batch)
        print(f"   Uploaded batch {i} to {i+batch_size}")
        
    print("✅ Ingestion Complete.")

else:
    # If we skipped ingestion, we MUST load the BM25 model from disk
    # so we can still run queries.
    if os.path.exists(bm25_filename):
        bm25.load(bm25_filename)
        print("✅ Skipped Ingestion. Loaded existing BM25 params from file.")
    else:
        print("⚠️ Warning: BM25 file not found. You might need to re-ingest if retrieval fails.")

2025-12-11 00:39:55,579 - INFO - Found credentials in environment variables.


✅ Skipped Ingestion. Loaded existing BM25 params from file.


In [7]:
# Cell 5: Setup Retrieval & Re-ranking Engines
from typing import List

# 1. Define the Bedrock Cohere Re-ranker Class
class BedrockCohereReranker:
    def __init__(self, region_name="us-east-1"):
        self.client = boto3.client("bedrock-runtime", region_name=region_name)
        self.model_id = "cohere.rerank-v3-5:0"

    def rerank(self, query: str, docs: List[str], top_n: int = 5):
        # Docs must be a list of strings for the API
        if not docs: return []
        
        request_body = {
            "query": query, 
            "documents": docs, 
            "top_n": top_n, 
            "api_version": 2
        }
        
        try:
            response = self.client.invoke_model(modelId=self.model_id, body=json.dumps(request_body))
            response_body = json.loads(response['body'].read())
            results = response_body.get("results", [])
            return results # Returns list of {'index': int, 'relevance_score': float}
        except Exception as e:
            print(f"⚠️ Rerank Error: {e}")
            # Fallback: return indices 0..top_n
            return [{"index": i, "relevance_score": 0.0} for i in range(min(len(docs), top_n))]

# Initialize the Reranker
reranker = BedrockCohereReranker()
print("✅ Cohere Re-ranker Initialized.")

# 2. Define the "Intelligent Retrieval" Function
# This combines Hybrid Search (Pinecone) + Re-ranking (Cohere)
def intelligent_retrieval(query: str) -> str:
    print(f"🔎 Searching for: '{query}'")
    
    # A. Hybrid Search in Pinecone (Top 25)
    dense_vec = embeddings.embed_query(query)
    # Note: If you want strict keyword matching, enable the line below:
    # sparse_vec = bm25.encode_queries(query) 
    
    results = index.query(
        vector=dense_vec,
        # sparse_vector=sparse_vec, # Uncomment if passing sparse values
        top_k=25,
        include_metadata=True
    )
    
    # Extract just the text from the matches
    raw_docs = [match['metadata']['text'] for match in results['matches']]
    
    if not raw_docs:
        return ""

    # B. Re-ranking (Filter 25 -> Top 5)
    rerank_results = reranker.rerank(query, raw_docs, top_n=5)
    
    # C. Format the Top 5 for the LLM
    top_docs_text = []
    for res in rerank_results:
        idx = res['index']
        top_docs_text.append(raw_docs[idx])
        
    return "\n\n".join(top_docs_text)

print("✅ Retrieval Logic Defined.")

2025-12-11 00:40:04,697 - INFO - Found credentials in environment variables.


✅ Cohere Re-ranker Initialized.
✅ Retrieval Logic Defined.


In [8]:
# Cell 5.5: Initialize anthropic.claude-3-5-haiku Model

from langchain_aws import ChatBedrock

# We use the US Cross-Region Inference Profile for Llama 3.1
llm = ChatBedrock(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",  ## us.meta.llama3-1-70b-instruct-v1:0
    client=bedrock_client, # We defined this client in Cell 4
    model_kwargs={"temperature": 0.1, "max_tokens": 512} # max_tokens": 2048
)

print("✅ anthropic.claude-3-5-haiku Model Initialized.")

✅ anthropic.claude-3-5-haiku Model Initialized.


In [9]:
# Cell 6: LLM Chain Setup
# 1. Define the Prompt
# We strictly tell the LLM to use ONLY the provided context.
prompt_template = """
You are an expert Insurance Assistant. Use the following pieces of retrieved context to answer the question.
If the answer is not in the context, just say that you don't know. Do not try to make up an answer.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

# 2. Define the Chain
# This pipeline does: Take Query -> Get Smart Context -> Format Prompt -> Run Llama 3 -> Parse String
rag_chain_final = (
    {
        "context": RunnableLambda(intelligent_retrieval), # Uses our Hybrid + Rerank function
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain (Production Ready) Created.")

# 3. Quick Sanity Check
# Let's run a simple test to make sure the chain flows correctly

test_q = "what is the name of company which provides this insurance? and give me address for this company. also give me contact details for this comapny"
#"What specific exclusions apply to loss caused by Subsidence?"
#"What is the deductible for Personal Property?"
#"from the document tell me in terms of payment what policys provide how much insurance back means in terms of money"


print(f"\n🧪 Sanity Check Query: '{test_q}'")
print("-" * 40)
print(rag_chain_final.invoke(test_q))

✅ RAG Chain (Production Ready) Created.

🧪 Sanity Check Query: 'what is the name of company which provides this insurance? and give me address for this company. also give me contact details for this comapny'
----------------------------------------
🔎 Searching for: 'what is the name of company which provides this insurance? and give me address for this company. also give me contact details for this comapny'


2025-12-11 00:40:13,543 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '1ccc8b58-6690-4b49-b94e-85a04d244916', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 10 Dec 2025 19:10:13 GMT', 'content-type': 'application/json', 'content-length': '43421', 'connection': 'keep-alive', 'x-amzn-requestid': '1ccc8b58-6690-4b49-b94e-85a04d244916', 'x-amzn-bedrock-invocation-latency': '87', 'x-amzn-bedrock-input-token-count': '30'}, 'RetryAttempts': 0}
2025-12-11 00:40:16,104 - INFO - Using Bedrock Invoke API to generate response


Based on the context provided, here are the details:

Company Name: SBI General Insurance Company Limited

Address: 
Fulcrum Building, 9th Floor, A & B Wing, Sahar Road, Andheri (East), Mumbai - 400099

Contact Details:
- Toll-free Number: 18001021111
- Email: customer.care@sbigeneral.in
- Website: www.sbigeneral.in

Additional Information:
- CIN (Corporate Identification Number): U66000MH2009PLC190546
- IRDAI Registration Number: 144


In [10]:
# In Cell 6, at the bottom - RUN THIS AGAIN after restart
test_q = "What specific exclusions apply to loss caused by Subsidence?"

print(f"\n🧪 Sanity Check Query: '{test_q}'")
print("-" * 40)
result = rag_chain_final.invoke(test_q)
print(result)



🧪 Sanity Check Query: 'What specific exclusions apply to loss caused by Subsidence?'
----------------------------------------
🔎 Searching for: 'What specific exclusions apply to loss caused by Subsidence?'


2025-12-11 00:46:09,667 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'bc7e0f54-25ba-445b-9c4f-54eae07090c8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 10 Dec 2025 19:16:09 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': 'bc7e0f54-25ba-445b-9c4f-54eae07090c8', 'x-amzn-bedrock-invocation-latency': '86', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}
2025-12-11 00:46:11,836 - INFO - Using Bedrock Invoke API to generate response


Based on the provided context, for subsidence, the following exclusions apply:

Subsidence is excluded when caused by:
a. Normal cracking, settlement or bedding down of new structures
b. The settlement or movement of made up ground
c. Coastal or river erosion
d. Defective design or workmanship or use of defective materials
e. Demolition, construction, structural alterations or repair of any property
f. Groundworks or excavations

These exclusions are specifically listed under section 6 of the context, which covers "Subsidence of the land on which Your Home Buildings stands, Landslide, Rockslide".


In [11]:
# In Cell 6, at the bottom - RUN THIS AGAIN after restart
test_q = "who is prasad?"

print(f"\n🧪 Sanity Check Query: '{test_q}'")
print("-" * 40)
result = rag_chain_final.invoke(test_q)
print(result)



🧪 Sanity Check Query: 'who is prasad?'
----------------------------------------
🔎 Searching for: 'who is prasad?'


2025-12-11 00:58:38,341 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'c278da52-5e15-41e2-836c-9af1521079a1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 10 Dec 2025 19:28:38 GMT', 'content-type': 'application/json', 'content-length': '43336', 'connection': 'keep-alive', 'x-amzn-requestid': 'c278da52-5e15-41e2-836c-9af1521079a1', 'x-amzn-bedrock-invocation-latency': '88', 'x-amzn-bedrock-input-token-count': '6'}, 'RetryAttempts': 0}
2025-12-11 00:58:40,460 - INFO - Using Bedrock Invoke API to generate response


Based on the provided context, I cannot find any specific information about who Prasad is. The context contains definitions related to insurance terminology like medical practitioners, mis-selling, nominees, and policy holders, but there is no mention of a person named Prasad. Therefore, I do not know who Prasad is from this context.


In [ ]:
# In Cell 6, at the bottom - RUN THIS AGAIN after restart
test_q = 'who is prasad?

print(f"\n🧪 Sanity Check Query: '{test_q}'")
print("-" * 40)
result = rag_chain_final.invoke(test_q)
print(result)


Object `Property` not found.

🧪 Sanity Check Query: 'who is prasad?'
----------------------------------------
🔎 Searching for: 'who is prasad?'


2025-12-11 01:05:35,309 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '15260c9f-c11f-43af-9edd-3d79b6d22024', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 10 Dec 2025 19:35:35 GMT', 'content-type': 'application/json', 'content-length': '43336', 'connection': 'keep-alive', 'x-amzn-requestid': '15260c9f-c11f-43af-9edd-3d79b6d22024', 'x-amzn-bedrock-invocation-latency': '75', 'x-amzn-bedrock-input-token-count': '6'}, 'RetryAttempts': 0}
2025-12-11 01:05:37,388 - INFO - Using Bedrock Invoke API to generate response


Based on the provided context, I cannot find any specific information about who Prasad is. The context contains definitions related to insurance terminology like medical practitioners, mis-selling, nominees, and policy holders, but there is no mention of a person named Prasad. Therefore, I do not know who Prasad is from this context.


In [13]:
# Cell 7: Create Evaluation Dataset for RAG (Based on Actual SBI Policy)
from langsmith import Client

# Initialize LangSmith client
ls_client = Client()

# Dataset name
dataset_name = "sbi-insurance-qa-eval"

# Evaluation Examples with REAL questions and answers from the PDF
eval_examples = [
    {
        "inputs": {"question": "What specific exclusions apply to loss caused by Subsidence?"},
        "outputs": {"expected_answer": "Subsidence losses are excluded if caused by: (a) normal cracking, settlement or bedding down of new structures, (b) settlement or movement of made up ground, (c) coastal or river erosion, (d) defective design or workmanship or use of defective materials, or demolition, construction, structural alterations or repair of any property, or groundworks or excavations"}
    },
    {
        "inputs": {"question": "What is the compensation amount for Personal Accident death cover?"},
        "outputs": {"expected_answer": "The Personal Accident Cover provides compensation of Rs 5,00,000 (Five Lakh) per person in the event of unfortunate death of either the policyholder or their spouse due to an insured peril"}
    },
    {
        "inputs": {"question": "Are acts of terrorism covered under this policy?"},
        "outputs": {"expected_answer": "Yes, acts of terrorism are covered as per the Terrorism Clause attached to the policy, with applicable exclusions and excess mentioned in that clause"}
    },
    {
        "inputs": {"question": "How much time do I have to submit a claim after noticing loss or damage?"},
        "outputs": {"expected_answer": "You must submit your claim in the claim form at the earliest opportunity, but within 30 days from the date you first notice the loss or damage"}
    },
    {
        "inputs": {"question": "Is earthquake damage covered as standard?"},
        "outputs": {"expected_answer": "Yes, earthquake, volcanic eruption, or other convulsions of nature are covered perils under Section 1 Fire and Allied Perils of the policy"}
    },
    {
        "inputs": {"question": "What is the maximum period for Loss of Rent coverage?"},
        "outputs": {"expected_answer": "The Loss of Rent coverage is available for the reasonable time required to repair the Home Building to make it fit for living. The maximum period of this cover is three years from the date the Home Building becomes unfit for living"}
    },
    {
        "inputs": {"question": "Are electrical appliances covered for breakdown?"},
        "outputs": {"expected_answer": "Electrical/electronic damage by over-running, short circuiting, arcing, self-heating or leakage of electricity is excluded under Section 1. However, Section 6 Breakdown of Domestic Electric Electronic Appliances provides optional coverage for unforeseen mechanical/electrical breakdown and accidental external damage"}
    },
    {
        "inputs": {"question": "What happens to the policy if my home remains unoccupied?"},
        "outputs": {"expected_answer": "Under the Burglary and Theft section, the policy will not make payment if loss or damage occurs while your home is Unoccupied (more than 30 consecutive days) unless the company was informed at the time of applying for insurance or prior to the home being unoccupied, and it is signified by an endorsement on the policy"}
    },
    {
        "inputs": {"question": "What is the built-in cover for Home Contents if I purchase Home Building cover?"},
        "outputs": {"expected_answer": "The policy has a built-in cover for General Contents equal to 20% of the Sum Insured for Home Building Cover, subject to a maximum of Rs 10 Lakh, provided both Home Building and Home Contents cover are opted for"}
    },
    {
        "inputs": {"question": "What is the sum insured restoration process after a claim?"},
        "outputs": {"expected_answer": "After payment of any loss, the policy shall be restored to the full original amount of Sum Insured. The policyholder must pay proportionate premium for the unexpired Policy Period from the date of loss, which can be deducted from the net claim amount"}
    },
    {
        "inputs": {"question": "What reports must be submitted to authorities after fire damage?"},
        "outputs": {"expected_answer": "In case of fire, explosion, implosion or lightning damage, you must give immediate report to the fire brigade of the local authority and the police"}
    },
    {
        "inputs": {"question": "What is covered under Burglary section for newly purchased contents?"},
        "outputs": {"expected_answer": "Newly purchased Contents purchased after commencement of the Policy are covered subject to maximum payment of 10% of the Section Sum Insured or Rs 20,000 whichever is less, duly supported by original purchase invoice/bill"}
    },
    {
        "inputs": {"question": "What is the Ambulance Expenses benefit limit?"},
        "outputs": {"expected_answer": "Ambulance Expenses benefit provides reimbursement up to Rs 5,000 per policy per year for reasonable and customary expenses incurred towards transportation by a registered ambulance service provider to a Hospital in case of an Emergency requiring hospitalization"}
    },
    {
        "inputs": {"question": "What are the architect and debris removal cost limits?"},
        "outputs": {"expected_answer": "The policy pays up to 5% of the claim amount for reasonable fees of architect, surveyor, consulting engineer and up to 2% of the claim amount for reasonable costs of removing debris from the site"}
    },
    {
        "inputs": {"question": "Are pre-existing diseases covered under Personal Accident section?"},
        "outputs": {"expected_answer": "No, any Pre-existing Disease or Disability arising out of Pre-existing Diseases or any complication arising therefrom is specifically excluded under the Personal Accident section"}
    }
]

# Create or Check if Dataset Exists
try:
    dataset = ls_client.read_dataset(dataset_name=dataset_name)
    print(f"✅ Dataset '{dataset_name}' already exists with {len(list(ls_client.list_examples(dataset_id=dataset.id)))} examples.")
except:
    print(f"⚙️ Creating new dataset: '{dataset_name}'...")
    dataset = ls_client.create_dataset(
        dataset_name=dataset_name,
        description="Evaluation dataset for SBI Home Insurance RAG with 15 realistic questions and accurate ground truth answers from policy document"
    )
    
    print(f"📝 Adding {len(eval_examples)} examples to dataset...")
    for example in eval_examples:
        ls_client.create_example(
            inputs=example["inputs"],
            outputs=example["outputs"],
            dataset_id=dataset.id
        )
    
    print(f"✅ Dataset created successfully with {len(eval_examples)} examples!")

print(f"\n🔗 View your dataset: https://smith.langchain.com/datasets/{dataset.id}")


⚙️ Creating new dataset: 'sbi-insurance-qa-eval'...
📝 Adding 15 examples to dataset...
✅ Dataset created successfully with 15 examples!

🔗 View your dataset: https://smith.langchain.com/datasets/cdbbfeab-3c59-4a87-b828-a2425c4d0512


In [14]:
# Cell 8: Run Evaluation on Dataset
from langsmith.evaluation import evaluate

# 1. Define a wrapper function for your RAG chain
# LangSmith evaluate() expects a function that takes a dict with "question" 
# and returns a dict with the answer
def rag_predict(inputs: dict) -> dict:
    """Wrapper function for RAG chain evaluation"""
    question = inputs["question"]
    answer = rag_chain_final.invoke(question)
    return {"answer": answer}

# 2. Define a simple evaluator
# This checks if the answer is relevant (not empty and not "I don't know")
def answer_not_empty(run, example):
    """Check if RAG provided a meaningful answer"""
    answer = run.outputs.get("answer", "").lower()
    
    # Check if answer is meaningful
    is_meaningful = (
        len(answer) > 20 and 
        "don't know" not in answer and
        answer.strip() != ""
    )
    
    return {
        "key": "answer_provided",
        "score": 1 if is_meaningful else 0
    }

# 3. Run the evaluation
print("🧪 Starting evaluation on dataset...")
print("This will run your RAG chain on all 15 questions and track results in LangSmith.\n")

results = evaluate(
    rag_predict,  # Your RAG function
    data=dataset_name,  # The dataset we just created
    evaluators=[answer_not_empty],  # Basic evaluator
    experiment_prefix="sbi-rag-eval",  # Name for this evaluation run
    description="Testing RAG system with hybrid search + reranking on SBI insurance questions",
    max_concurrency=1  # Run one at a time to avoid API limits
)

print("\n✅ Evaluation Complete!")
print(f"📊 Results Summary:")
print(f"   - Total Questions: {results.summary['example_count']}")
print(f"   - Answers Provided: {results.summary.get('answer_provided', {}).get('score', 0) * 100:.1f}%")
print(f"\n🔗 View detailed results: {results.experiment_url}")


🧪 Starting evaluation on dataset...
This will run your RAG chain on all 15 questions and track results in LangSmith.

View the evaluation results for experiment: 'sbi-rag-eval-3fb11671' at:
https://smith.langchain.com/o/361d1350-8788-45f2-83b5-4450425627e0/datasets/cdbbfeab-3c59-4a87-b828-a2425c4d0512/compare?selectedSessions=77099f6f-31ed-4079-8fcf-f76faf1e0cf9




0it [00:00, ?it/s]

🔎 Searching for: 'Are pre-existing diseases covered under Personal Accident section?'


2025-12-11 01:26:25,134 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'bd3d2b89-77db-4a25-a8e5-4d98a2098c9f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 10 Dec 2025 19:56:25 GMT', 'content-type': 'application/json', 'content-length': '43377', 'connection': 'keep-alive', 'x-amzn-requestid': 'bd3d2b89-77db-4a25-a8e5-4d98a2098c9f', 'x-amzn-bedrock-invocation-latency': '80', 'x-amzn-bedrock-input-token-count': '11'}, 'RetryAttempts': 0}
2025-12-11 01:26:27,369 - INFO - Using Bedrock Invoke API to generate response


🔎 Searching for: 'What are the architect and debris removal cost limits?'


...

13it [02:06, 11.99s/it]

🔎 Searching for: 'What specific exclusions apply to loss caused by Subsidence?'


2025-12-11 01:28:31,096 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'e5d65ed1-9e42-4cfd-8745-5924f6913a22', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 10 Dec 2025 19:58:31 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': 'e5d65ed1-9e42-4cfd-8745-5924f6913a22', 'x-amzn-bedrock-invocation-latency': '75', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}
2025-12-11 01:28:31,779 - INFO - Using Bedrock Invoke API to generate response
2025-12-11 01:28:32,330 - WARNING - Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('422 Client Error: unknown for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unprocessable entity: error reading multipart data: multipart: NextPart: EOF"}\n')
15it [02:10,  8.72s/it]



✅ Evaluation Complete!
📊 Results Summary:


AttributeError: 'ExperimentResults' object has no attribute 'summary'

In [ ]:
# Cell 5 (Updated): Setup Retrieval & Re-ranking with Tracing

## this cell for to check the context/chunk recive to re-ranker in Langsmith

from typing import List
from langsmith import traceable

# 1. Define the Bedrock Cohere Re-ranker Class
class BedrockCohereReranker:
    def __init__(self, region_name="us-east-1"):
        self.client = boto3.client("bedrock-runtime", region_name=region_name)
        self.model_id = "cohere.rerank-v3-5:0"

    @traceable(name="Cohere Reranker")  # This makes it visible in LangSmith!
    def rerank(self, query: str, docs: List[str], top_n: int = 5):
        """Rerank documents using Cohere Rerank v3.5"""
        if not docs: 
            return []
        
        request_body = {
            "query": query, 
            "documents": docs, 
            "top_n": top_n, 
            "api_version": 2
        }
        
        try:
            response = self.client.invoke_model(
                modelId=self.model_id, 
                body=json.dumps(request_body)
            )
            response_body = json.loads(response['body'].read())
            results = response_body.get("results", [])
            
            # Log metadata for LangSmith
            print(f"✅ Reranked {len(docs)} -> {len(results)} docs")
            return results
            
        except Exception as e:
            print(f"⚠️ Rerank Error: {e}")
            # Fallback: return indices 0..top_n
            return [{"index": i, "relevance_score": 0.0} for i in range(min(len(docs), top_n))]


# Initialize the Reranker
reranker = BedrockCohereReranker()
print("✅ Cohere Re-ranker Initialized.")


# 2. Define the "Intelligent Retrieval" Function with Tracing
@traceable(name="Hybrid Retrieval + Reranking")  # This wraps the entire retrieval process!
def intelligent_retrieval(query: str) -> str:
    """
    Performs hybrid search in Pinecone followed by Cohere reranking.
    Returns top 5 most relevant document chunks.
    """
    print(f"🔎 Searching for: '{query}'")
    
    # A. Hybrid Search in Pinecone (Top 25)
    dense_vec = embeddings.embed_query(query)
    
    results = index.query(
        vector=dense_vec,
        top_k=25,
        include_metadata=True
    )
    
    # Extract text from matches
    raw_docs = [match['metadata']['text'] for match in results['matches']]
    
    if not raw_docs:
        return ""

    # B. Re-ranking (Filter 25 -> Top 5)
    # The @traceable decorator on rerank() will show this step in LangSmith
    rerank_results = reranker.rerank(query, raw_docs, top_n=5)
    
    # C. Format the Top 5 for the LLM
    top_docs_text = []
    for res in rerank_results:
        idx = res['index']
        top_docs_text.append(raw_docs[idx])
        
    return "\n\n".join(top_docs_text)


print("✅ Retrieval Logic Defined with Tracing.")


In [ ]:
# Quick test to see tracing
test_q = "What are the architect and debris removal cost limits?"
result = rag_chain_final.invoke(test_q)
print(result)


In [13]:
# Cell 7: Final Evaluation Run
import pandas as pd

# 1. Define the Hard Questions
test_questions = [
    "What specific exclusions apply to loss caused by Subsidence?", 
    "What is the deductible for Personal Property?",
    "What are the specific exclusions for Riot, strikes, or malicious damages?"
]

print("🚀 Running Final Evaluation on Test Set...")
print("-" * 50)

results = []

for q in test_questions:
    print(f"Asking: {q}")
    try:
        # Run the robust chain
        answer = rag_chain_final.invoke(q)
        
        # Save result
        results.append({
            "Question": q,
            "AI Answer": answer.strip(),
            "Status": "✅ Success"
        })
    except Exception as e:
        results.append({
            "Question": q,
            "AI Answer": f"ERROR: {e}",
            "Status": "❌ Failed"
        })

# 2. Display Results in a Clean Table
df = pd.DataFrame(results)

print("\n" + "="*60)
print("🏆 FINAL PROJECT ACCURACY REPORT")
print("="*60)

# Print full details for verification
for i, row in df.iterrows():
    print(f"\nQ{i+1}: {row['Question']}")
    print(f"A: {row['AI Answer']}")
    print("-" * 40)

🚀 Running Final Evaluation on Test Set...
--------------------------------------------------
Asking: What specific exclusions apply to loss caused by Subsidence?
🔎 Searching for: 'What specific exclusions apply to loss caused by Subsidence?'


2025-12-06 16:30:42,721 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '80e646a4-7d00-42c3-b10f-33d176fed6c3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 11:00:23 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': '80e646a4-7d00-42c3-b10f-33d176fed6c3', 'x-amzn-bedrock-invocation-latency': '95', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}
2025-12-06 16:31:17,781 - INFO - Using Bedrock Invoke API to generate response


Asking: What is the deductible for Personal Property?
🔎 Searching for: 'What is the deductible for Personal Property?'


2025-12-06 16:31:27,586 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '0574e333-07bf-451a-a7a8-964937c2869c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 11:01:22 GMT', 'content-type': 'application/json', 'content-length': '43328', 'connection': 'keep-alive', 'x-amzn-requestid': '0574e333-07bf-451a-a7a8-964937c2869c', 'x-amzn-bedrock-invocation-latency': '78', 'x-amzn-bedrock-input-token-count': '9'}, 'RetryAttempts': 0}
2025-12-06 16:31:39,071 - INFO - Using Bedrock Invoke API to generate response


Asking: What are the specific exclusions for Riot, strikes, or malicious damages?
🔎 Searching for: 'What are the specific exclusions for Riot, strikes, or malicious damages?'


2025-12-06 16:31:51,091 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '5f507c34-7f0a-4b1f-896b-f02b37e2ddee', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 11:01:42 GMT', 'content-type': 'application/json', 'content-length': '43437', 'connection': 'keep-alive', 'x-amzn-requestid': '5f507c34-7f0a-4b1f-896b-f02b37e2ddee', 'x-amzn-bedrock-invocation-latency': '73', 'x-amzn-bedrock-input-token-count': '16'}, 'RetryAttempts': 0}
2025-12-06 16:32:01,671 - INFO - Using Bedrock Invoke API to generate response



🏆 FINAL PROJECT ACCURACY REPORT

Q1: What specific exclusions apply to loss caused by Subsidence?
A: According to the context, the specific exclusions that apply to loss caused by Subsidence of the land on which the home building stands are:

a. normal cracking, settlement or bedding down of new structures,
b. the settlement or movement of made up ground,
c. coastal or river erosion,
d. defective design or workmanship or use of defective materials, or demolition, construction, structural alterations or repair of any property, or groundworks or excavations.
----------------------------------------

Q2: What is the deductible for Personal Property?
A: The deductible for Personal Property is not explicitly mentioned in the context. However, it does mention deductibles for Jewellery & Valuables (5% of the claim amount subject to a minimum of Rs 2500) and portable equipment's (5% of claim amount subject to a minimum of Rs 1000).
----------------------------------------

Q3: What are the sp

In [14]:
# Cell 8: (Optional) Re-Run Ragas Metrics
# Note: This takes 1-2 minutes and costs a small amount of API usage.

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.run_config import RunConfig
from datasets import Dataset

# 1. Prepare Data from the results you just generated
questions_list = [row['Question'] for row in results]
answers_list = [row['AI Answer'] for row in results]
ground_truths = [
    "Normal cracking, settlement of new structures, movement of made up ground, coastal erosion, defective design.",
    "The document does not state a specific deductible for 'Personal Property', only for Jewellery and Portables.",
    "Temporary or permanent dispossession by government order, or unlawful occupation by any person."
]

# We need to fetch the contexts again manually for Ragas
contexts_list = []
print("fetching contexts for evaluation...")
for q in questions_list:
    # Quick re-fetch of the text the LLM saw
    retrieved_text = intelligent_retrieval(q)
    contexts_list.append([retrieved_text])

data_samples = {
    "question": questions_list,
    "answer": answers_list,
    "contexts": contexts_list,
    "ground_truth": ground_truths
}

ragas_dataset = Dataset.from_dict(data_samples)

# 2. Configure Ragas with Safety Mode (Sequential)
ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)
safe_config = RunConfig(max_workers=1, timeout=120, max_retries=3)

# 3. Run
print("👨‍⚖️ Calculating Final Scores...")
eval_results = evaluate(
    ragas_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    run_config=safe_config,
    raise_exceptions=False
)

print("\n" + "="*50)
print("🏆 OFFICIAL RAGAS SCORECARD")
print("="*50)
print(eval_results)

fetching contexts for evaluation...
🔎 Searching for: 'What specific exclusions apply to loss caused by Subsidence?'


2025-12-06 16:36:10,511 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '9bbd495c-b7a9-4f53-8227-d8358966bec3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 11:05:29 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': '9bbd495c-b7a9-4f53-8227-d8358966bec3', 'x-amzn-bedrock-invocation-latency': '79', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}


🔎 Searching for: 'What is the deductible for Personal Property?'


2025-12-06 16:36:20,926 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '7df64f13-5668-44b7-aadc-8bc414676274', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 11:06:17 GMT', 'content-type': 'application/json', 'content-length': '43328', 'connection': 'keep-alive', 'x-amzn-requestid': '7df64f13-5668-44b7-aadc-8bc414676274', 'x-amzn-bedrock-invocation-latency': '84', 'x-amzn-bedrock-input-token-count': '9'}, 'RetryAttempts': 0}


🔎 Searching for: 'What are the specific exclusions for Riot, strikes, or malicious damages?'


2025-12-06 16:36:27,671 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '45222c46-ecb4-40e1-b43b-e07e1dcd5813', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 11:06:26 GMT', 'content-type': 'application/json', 'content-length': '43437', 'connection': 'keep-alive', 'x-amzn-requestid': '45222c46-ecb4-40e1-b43b-e07e1dcd5813', 'x-amzn-bedrock-invocation-latency': '72', 'x-amzn-bedrock-input-token-count': '16'}, 'RetryAttempts': 0}
C:\Users\DELL\AppData\Local\Temp\ipykernel_1600\1291553097.py:38: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(llm)
C:\Users\DELL\AppData\Local\Temp\ipykernel_1600\1291553097.py:39: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in 

👨‍⚖️ Calculating Final Scores...


Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]2025-12-06 16:36:34,386 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 16:36:53,511 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 16:37:13,642 - ERROR - Exception raised in Job[0]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Evaluating:  11%|█         | 1/9 [00:39<05:16, 39.51s/it]2025-12-06 16:37:13,691 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 16:37:13,696 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 16:37:13,701 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 16:37:42,161 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '05eef54a-6587-4f86-a5e2-7e66d57a2745', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 11:07:41 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-al


🏆 OFFICIAL RAGAS SCORECARD
{'faithfulness': 1.0000, 'answer_relevancy': 0.9556, 'context_precision': 1.0000}
